In [1]:
import tensorflow as tf
from transformers import T5Tokenizer, TFT5ForConditionalGeneration
import pandas as pd
import numpy as np
import seaborn as sns
import sklearn as sk
import matplotlib.pyplot as plt
import tensorflow as tf

2025-03-01 14:41:51.587282: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-01 14:41:51.716453: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740820311.780608    7233 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740820311.800140    7233 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-01 14:41:51.915997: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
model_name = "t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = TFT5ForConditionalGeneration.from_pretrained(model_name)


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
I0000 00:00:1740820320.634014    7233 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 5315 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3070 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6
All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditio

In [3]:
def shift_right(input_ids, pad_token_id, decoder_start_token_id):
    """
    Shifts input ids to the right by one position.
    The first token becomes the decoder_start_token_id.
    """
    batch_size = tf.shape(input_ids)[0]
    start_tokens = tf.fill([batch_size, 1], decoder_start_token_id)
    shifted_input_ids = tf.concat([start_tokens, input_ids[:, :-1]], axis=1)
    return shifted_input_ids

class MultiTaskT5(tf.keras.Model):
    def __init__(self, model_name, num_emotion_labels, num_sentiment_labels):
        super(MultiTaskT5, self).__init__()
        self.t5 = TFT5ForConditionalGeneration.from_pretrained(model_name)
        self.dropout = tf.keras.layers.Dropout(0.1)
        self.emotion_classifier = tf.keras.layers.Dense(
            num_emotion_labels, activation='softmax', name="emotion_classifier"
        )
        self.sentiment_classifier = tf.keras.layers.Dense(
            num_sentiment_labels, activation='softmax', name="sentiment_classifier"
        )

    def call(self, inputs, task, training=False, labels=None):
        if task in ['emotion', 'sentiment']:
            encoder_outputs = self.t5.encoder(
                input_ids=inputs['input_ids'],
                attention_mask=inputs['attention_mask'],
                training=training
            )
            pooled_output = tf.reduce_mean(encoder_outputs.last_hidden_state, axis=1)
            pooled_output = self.dropout(pooled_output, training=training)
            if task == 'emotion':
                return self.emotion_classifier(pooled_output)
            else:
                return self.sentiment_classifier(pooled_output)
        elif task == 'summary':
            if labels is None:
                outputs = self.t5(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    training=training
                )
                return outputs.logits
            else:
                pad_token_id = self.t5.config.pad_token_id
                decoder_start_token_id = self.t5.config.decoder_start_token_id
                decoder_input_ids = shift_right(labels, pad_token_id, decoder_start_token_id)
                outputs = self.t5(
                    input_ids=inputs['input_ids'],
                    attention_mask=inputs['attention_mask'],
                    decoder_input_ids=decoder_input_ids,
                    training=training
                )
                return outputs.logits
        else:
            raise ValueError("Unsupported task type. Use 'emotion', 'sentiment', or 'summary'.")

In [4]:
model_name = "t5-small"
num_emotion_labels = 6
num_sentiment_labels = 4  # Updated to 4 output labels for sentiment
model = MultiTaskT5(model_name, num_emotion_labels, num_sentiment_labels)
tokenizer = T5Tokenizer.from_pretrained(model_name)
        


All PyTorch model weights were used when initializing TFT5ForConditionalGeneration.

All the weights of TFT5ForConditionalGeneration were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [5]:
# Load weights saved in H5 format.
model.load_weights('./results/multi_task_T5_tf_weights')


In [6]:
# Load the tokenizer from the saved directory.
tokenizer = T5Tokenizer.from_pretrained("./results/multi_task_T5_tf/")


In [7]:
sentiment_mapping = {3: 'positive', 2: 'neutral', 1: 'negative', 0: 'irrelevant'}

emotion_mapping = {
    0: "anger", 
    1: "fear", 
    2: "joy", 
    3: "love", 
    4: "sadness", 
    5: "surprise"
}

def decode_sentiment(label_code):
    return sentiment_mapping.get(label_code, "Unknown")

def decode_emotion(label_code):
    return emotion_mapping.get(label_code, "Unknown")

In [8]:
def predict(text, task):
    if task in ['emotion', 'sentiment']:
        # Tokenize the input with a moderate max_length.
        inputs = tokenizer(text, return_tensors="tf", max_length=128, truncation=True, padding="max_length")
        logits = model(inputs, task=task, training=False)
        # Get the predicted label index.
        pred_idx = int(tf.argmax(logits, axis=1).numpy()[0])
        # Decode the prediction using our mapping functions.
        if task == "sentiment":
            return decode_sentiment(pred_idx)
        elif task == "emotion":
            return decode_emotion(pred_idx)
    
    elif task == 'summary':
        # Prepend the summarization prefix as required by T5.
        input_text = "summarize: " + text
        # Tokenize with a longer max_length for summarization inputs.
        inputs = tokenizer(input_text, return_tensors="tf", max_length=512, truncation=True, padding="max_length")
        # Use the underlying T5 model's generate function.
        generated_ids = model.t5.generate(
            inputs["input_ids"],
            attention_mask=inputs["attention_mask"],
            max_length=150,
            num_beams=4,
            early_stopping=True
        )
        # Decode the generated token ids to text.
        summary = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
        return summary
    
    else:
        raise ValueError("Unsupported task type. Use 'emotion', 'sentiment', or 'summary'.")


In [15]:
# Test sentiment classification
sentiment_result = predict("i fell bad", task="sentiment")
print("Sentiment Prediction:", sentiment_result)

# Test emotion classification
emotion_result = predict("smart guy, i would surely hire", task="emotion")
print("Emotion Prediction:", emotion_result)

# Test summarization
summary_result = predict("This is an example text that we want to summarize.", task="summary")
print("Summary Output:", summary_result)


Sentiment Prediction: negative
Emotion Prediction: joy
Summary Output: This is an example text that we want to summarize.


In [13]:
pred_data = pd.read_json('./comments.json')
pred_data['comments'][:100]

0     Go to https://ground.news/bog to gain deeper i...
1     I cannot believe that I wasted 40 minutes of m...
2     You're actually really close to getting it to ...
3     bro i literally went down this rabbit hole THI...
4     I like the approach you have toward learning s...
                            ...                        
95    By the way, there is a problem you gonna encou...
96    If you need to search something in a command o...
97    I'm proud of this man's documentation of his t...
98    You did it! When you see AMD GPU in your windo...
99    More than 15 years ago I ran a server farm of ...
Name: comments, Length: 100, dtype: object

In [19]:
for comment in list(pred_data['comments'][:100]):
    print(predict(comment, task="sentiment"))

    
    

neutral
neutral
neutral
positive
positive
positive
negative
positive
positive
positive
neutral
positive
neutral
negative
positive
neutral
negative
positive
neutral
neutral
neutral
positive
neutral
positive
positive
positive
neutral
positive
positive
positive
neutral
positive
irrelevant
positive
irrelevant
positive
positive
neutral
positive
positive
neutral
neutral
neutral
irrelevant
neutral
positive
positive
neutral
irrelevant
neutral
neutral
negative
positive
positive
neutral
positive
positive
neutral
positive
positive
negative
positive
neutral
positive
neutral
neutral
neutral
positive
neutral
irrelevant
neutral
neutral
positive
neutral
neutral
irrelevant
positive
negative
negative
negative
positive
positive
positive
neutral
neutral
negative
positive
positive
negative
irrelevant
neutral
negative
neutral
irrelevant
neutral
negative
neutral
positive
neutral
negative


In [20]:
for comment in list(pred_data['comments'][:100]):
    print(predict(comment, task="emotion"))

    

joy
sadness
sadness
joy
joy
joy
sadness
joy
joy
joy
sadness
joy
joy
sadness
joy
joy
sadness
joy
sadness
sadness
sadness
joy
joy
joy
joy
joy
joy
joy
joy
joy
joy
joy
joy
joy
joy
joy
joy
sadness
joy
joy
joy
joy
sadness
anger
joy
joy
joy
joy
joy
joy
joy
sadness
joy
joy
joy
joy
joy
joy
joy
joy
sadness
joy
joy
joy
sadness
joy
sadness
joy
sadness
joy
sadness
joy
joy
joy
sadness
sadness
joy
sadness
sadness
sadness
joy
joy
joy
joy
sadness
sadness
joy
joy
sadness
sadness
joy
sadness
joy
sadness
sadness
sadness
sadness
joy
sadness
sadness


In [27]:
summ_comments=''
for comment in list(pred_data['comments'][:10]):
    summ_comments += comment

In [28]:
# for comment in list(pred_data['comments'][:10]):
print(predict(summ_comments, task="summary"))


InspiringIm still waiting for the "I don't really need this" stagewatching this from an IT perspective really gives you an insight on how you might think something are just a given for you isn't the case for a normal person.


In [30]:

summ_comments

'Go to https://ground.news/bog to gain deeper insights and keep up with current events. Sign up through my link for 50% off unlimited access to their Vantage Plan!I cannot believe that I wasted 40 minutes of my life watching a guy wasting a couple of days of his life following random tutorials on the internet. 10/10 liked it.You\'re actually really close to getting it to work :) The way passing through a secondary GPU works is that it basically gives the VM software (in this case qemu) direct access to the graphics card - that\'s why you couldn\'t see anything in linux when you only had your dGPU plugged in to your monitor. \n\nAny output from the dGPU is going to display nothing until qemu starts up windows - so your monitor will automatically sync to where it gets video first, in this case, linux using your iGPU. By switching which graphics card output you are displaying on your monitor, you will be switching between seeing linux and windows.\n\nAlso, the reason you were still seeing